In [ ]:
from meeko import MoleculePreparation
from vina import Vina
from rdkit import Chem
from meeko import PDBQTMolecule
from meeko import PDBQTWriterLegacy

In [ ]:
v = Vina(sf_name='vina', cpu=8)
v.set_receptor('Data/9f6a.pdbqt')
v.set_ligand_from_file('Data/lig-1.pdbqt')

docking_box = {"center": [136.733, 172.819, 99.189], "box_size": [11.69, 7.09, 7.60]}
v.compute_vina_maps(**docking_box)

v.dock(exhaustiveness=8, n_poses=5)

In [4]:
preparator = MoleculePreparation()

In [ ]:
test_lig = Chem.SDMolSupplier('Data/ligands.sdf')[0]
test_lig

In [85]:
test_lig = Chem.AddHs(test_lig)

In [ ]:
Chem.rdDistGeom.EmbedMolecule(test_lig)

In [ ]:
test_lig

In [88]:
mol_setups = preparator.prepare(test_lig)

In [ ]:
for setup in mol_setups:
    pdbqt_string, is_ok, error_msg = PDBQTWriterLegacy.write_string(setup)
    if is_ok:
        print(pdbqt_string, end = "")

In [91]:
ligands = Chem.SDMolSupplier('Data/ligands.sdf')

In [ ]:
len(ligands)

In [ ]:
for i, mol in enumerate(ligands):
    if mol is None:
        continue
    if i >= 100:
        break
    print(ligands[i].GetProp('_Name'))

In [ ]:
from rdkit.Chem import SDMolSupplier
 
def process_ligands_in_batches(input_sdf_file, batch_size):
    suppl = SDMolSupplier(input_sdf_file)
 
    batch = []
    for idx, mol in enumerate(suppl):
        if mol is not None:
            batch.append(mol)
 
        # Process and flush the batch when the batch size is reached
        if len(batch) == batch_size:
            process_batch(batch, idx // batch_size)
            batch.clear()  # Clear the batch after processing
 
    # Process any remaining ligands in the final batch
    if batch:
        process_batch(batch, idx // batch_size)
 
def process_batch(batch, batch_idx):
    # Convert batch to PDBQT or perform docking in a single step
    print(f"Processing batch {batch_idx + 1} with {len(batch)} ligands.")
    # Replace with actual conversion and docking logic
 
# Example usage
input_sdf_file = 'ligands.sdf'
batch_size = 100  # Adjust batch size for performance and memory
 
process_ligands_in_batches(input_sdf_file, batch_size)

In [11]:
import os

In [12]:
if not os.path.exists('Output'):
    os.makedirs('Output')

In [8]:
from openbabel import pybel
from openbabel import openbabel

In [ ]:
mol_iter = pybel.readfile(filename = 'Data/ligands_10.sdf', format = 'sdf')

In [ ]:
type(mol_iter)

In [ ]:
for mol in mol_iter:
    print(mol.molwt)

In [17]:
from openbabel import openbabel
from openbabel import pybel

# Input and output file paths
input_smi = "Data/ligands_10_2.smi"
output_sdf = "output_molecules4.pdbqt"

converter = openbabel.OBConversion()
converter.SetInAndOutFormats("sdf", "pdbqt")

# Desired pH
pH = 7.0

# Open the output file
with open(output_sdf, "w") as outfile:
    # Read molecules from the SDF file
    for mol in pybel.readfile("smi", input_smi):
        # Convert to OBMol for direct manipulation
        obmol = mol.OBMol

        # Adjust hydrogens at the specified pH
        obmol.AddHydrogens(False, True, pH)
    
        # Convert the adjusted molecule back to Pybel format
        adjusted_mol = pybel.Molecule(obmol)

        adjusted_mol.make3D()

        # Write output
        outfile.write(adjusted_mol.write("pdbqt"))


In [ ]:
import ringtail as rtc
from joblib import Parallel, delayed
from rdkit import Chem
from meeko import MoleculePreparation, PDBQTWriterLegacy
from vina import Vina
from openbabel import openbabel, pybel
from re import split, sub, MULTILINE

In [4]:
# Ligand library file in .sdf format.
sdf_file = "Data/ligands_10.sdf"

# Ligand library file in .smi format.
smi_file = "Data/ligands_10.smi"
smi_file_no_header = "Data/ligands_10_2.smi"

suppl = Chem.SDMolSupplier(sdf_file) # SDMolSupplier is an iterator. There's also an experimental 'MultithreadedSDMolSupplier' that may be faster.
suppl2 = Chem.SmilesMolSupplier(smi_file, delimiter="\t") # Iterator for the .smi ligand file.
# NOTE: Suspicion is that the pybel.readfile() iterator causes the pickling issue with joblib.
suppl4 = pybel.readfile("smi", smi_file_no_header) # Test pybel iterator for the ligand batching.

In [ ]:
# Text mol name extraction from rdkit mol object 

suppl2 = Chem.SmilesMolSupplier(smi_file, delimiter="\t") # Iterator for the .smi ligand file.
preparator = MoleculePreparation()

for mol in suppl2:

    mol_name = mol.GetProp('_Name')

    mol = Chem.AddHs(mol)
    Chem.rdDistGeom.EmbedMolecule(mol)
      
    mol_setups = preparator.prepare(mol)
    for setup in mol_setups:
        pdbqt_string, is_ok, error_msg = PDBQTWriterLegacy.write_string(setup)
        if is_ok:
            modified_pdbqt = f"REMARK Name = {mol_name.strip()}\n{pdbqt_string}"
            print(modified_pdbqt)

In [ ]:
with open(smi_file_no_header, "r") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) >= 2:  # Ensure the line has both SMILES and name.
                smiles, mol_name = line, parts[1]
                print((smiles, mol_name))  # Append a tuple of SMILES and name.

In [ ]:
# OK
preparator = MoleculePreparation()

for mol in suppl2:

    mol = Chem.AddHs(mol)
    Chem.rdDistGeom.EmbedMolecule(mol)
        
    mol_setups = preparator.prepare(mol)
    for setup in mol_setups:
        pdbqt_string, is_ok, error_msg = PDBQTWriterLegacy.write_string(setup)
        if is_ok:
            print(pdbqt_string)

In [ ]:
# OK
preparator = MoleculePreparation()

for mol in suppl2:

    mol = Chem.AddHs(mol)
    Chem.rdDistGeom.EmbedMolecule(mol)
        
    mol_setups = preparator.prepare(mol)
    for setup in mol_setups:
        pdbqt_string, is_ok, error_msg = PDBQTWriterLegacy.write_string(setup)
        if is_ok:
            print(pdbqt_string)

In [ ]:
preparator = MoleculePreparation()

for mol in pybel.readfile("smi", smi_file_no_header):

    obmol = mol.OBMol

    # Add hydrogens and consider protonation states.
    # AddHydrogen() params: (bool polaronly=false, bool correctForPH=false, double pH=7.4)
    obmol.AddHydrogens(False, False, 7.4)

    # Convert the Open Babel molecule back to pybel molecule.
    mol = pybel.Molecule(obmol)

    mol.make3D()
    #mol.localopt() # Coordinate improvement.

    pybel_string = mol.write(format = "pdbqt")

    print(pybel_string)

In [ ]:
from re import split, sub, MULTILINE

suppl4 = pybel.readfile("smi", smi_file_no_header)

for mol in suppl4:
    smiles_string = split("\t", mol.write(format = "smi"))[0]
    mol_name = split("\t", mol.write(format = "smi"))[1]
    obmol = mol.OBMol
    
    # Add hydrogens and consider protonation states
    obmol.AddHydrogens(False, False, 7.4)
    
    # Convert the Open Babel molecule back to pybel molecule
    mol = pybel.Molecule(obmol)
    mol.make3D()
    
    # Write PDBQT and insert custom REMARK fields
    pybel_string = mol.write(format = "pdbqt")
    # modified_pdbqt = f"REMARK  Name = {mol_name.strip()}\nREMARK  SMILES = {smiles_string.strip()}\n{pybel_string}"
    modified_pdbqt = f"REMARK  SMILES = {smiles_string.strip()}\n{pybel_string}"
    modified_pdbqt = sub(r'^REMARK\s+Name\s*=.*$', f'REMARK Name = {mol_name.strip()}', modified_pdbqt, flags=MULTILINE) # Replaces the Name remark with the molecule name.
    print(modified_pdbqt)

In [127]:
def molecule_prep2(idx, mol):
    try:
        # Convert the pybel molecule to Open Babel molecule.
        obmol = smiles_to_obmol(mol)

        # Add hydrogens and consider protonation states.
        obmol.AddHydrogens(False, True, 7.4)

        # Convert the Open Babel molecule back to pybel molecule.
        mol = pybel.Molecule(obmol)

        mol.make3D()

        return idx, mol.write(format = "pdbqt")
    except Exception as e:
        return idx, None, str(e)

In [124]:
from openbabel import pybel
from openbabel import openbabel

def smiles_to_obmol(smiles_string):
    # Create an OpenBabel conversion object
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("smi", "mol")
    
    # Create an OBMol object
    obmol = openbabel.OBMol()
    
    # Convert SMILES to OBMol
    obConversion.ReadString(obmol, smiles_string)
    
    return obmol

In [ ]:
batch = []
suppl4 = pybel.readfile("smi", smi_file_no_header)
for idx, mol in enumerate(suppl4):
    if mol is not None:


        batch.append(mol.write(format = "smi"))

    if len(batch) == 10:

        converted_batch = Parallel(n_jobs=2)(
            delayed(molecule_prep2)(idx, mol) # Delayed call for molecule_prep2() once below evaluation to done.
            for idx, mol in enumerate(batch)
            if mol is not None
        )

In [ ]:
# Extracting the SMILES string from the rdkit mol object
smiles_string = Chem.MolToSmiles(mol)

In [ ]:
pybel.informats

In [2]:
from scrubber import Scrub
from rdkit import Chem

In [ ]:
scrub = Scrub(ph_low=2, ph_high=11)

smiles_list = [
    "CC(C)NC(=O)C1=CC=CO1",
    "COCCNC(=O)C(C)(C)Cl",
    "COC(CN)CN1CCCC1",
    "CC(C)COC(=O)CC(Cl)Cl",
    "CSCC1=CC=C(CF)C=C1",
    "CCN(CC)C(=O)C=CCO",
    "CC[C@H]1CC(NCCO)CN1",
    "CCCCNC1CC(CCl)C1",
    "CCOC(=O)NCCC(C)C"
]


for smiles in smiles_list:
    for mol_state in scrub(Chem.MolFromSmiles(smiles)):
        print("SMILES: ", smiles, Chem.MolToSmiles(mol_state), "Conformers: %d" % mol_state.GetNumConformers())

In [ ]:
mol = Chem.MolFromSmiles("C=CCN1CC23CCC2(COC3)C1")
mol_name = "mol1"
if mol is None:
    raise ValueError(f"Invalid SMILES string: {smiles}")

# Assign the molecule name to the RDKit object.
mol.SetProp('_Name', mol_name)

# Prepare the ligand with Meeko.
preparator = MoleculePreparation()

variants = []

# Scrubber handles protonation states, 3D coordinates, and tautomers.
for mol_index, mol_state in enumerate(scrub(mol)):
    variant_mol_name = f"{mol_name}-{mol_index}"
    
    mol_setups = preparator.prepare(mol_state)

    for setup in mol_setups:
        pdbqt_string, is_ok, error_msg = PDBQTWriterLegacy.write_string(setup)
        if is_ok:
            modified_pdbqt = f"REMARK Name = {variant_mol_name.strip()}\n{pdbqt_string}"
            variants.append((variant_mol_name, modified_pdbqt))
print(variants)

In [ ]:
from rdkit.Chem import AllChem as Chem


smiles = "C=CCN1CC23CCC2(COC3)C1"

mol = Chem.MolFromSmiles(smiles)

fragments = Chem.GetMolFrags(mol, asMols=True)

print(len(fragments))

# print(Chem.MolToSmiles(fragments[0]))
# main_fragment = max(fragments, key=lambda m: m.GetNumAtoms())
# print(Chem.MolToSmiles(main_fragment))

In [ ]:
from meeko import MoleculePreparation, PDBQTWriterLegacy
from scrubber import Scrub
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.MolStandardize.rdMolStandardize import LargestFragmentChooser

scrub = Scrub(
        ph_low = 2,
        ph_high = 11,
    )

# smiles_list = [
#     "CC(C)NC(=O)C1=CC=CO1",
#     "COCCNC(=O)C(C)(C)Cl",
#     "COC(CN)CN1CCCC1",
#     "CC(C)COC(=O)CC(Cl)Cl",
#     "CSCC1=CC=C(CF)C=C1",
#     "CCN(CC)C(=O)C=CCO",
#     "CC[C@H]1CC(NCCO)CN1",
#     "CCCCNC1CC(CCl)C1",
#     "CCOC(=O)NCCC(C)C"
# ]

smiles_list = [
    "CCCCCSCC[N+](C)(C)C.[Br-]"
    
]

i = 0

for smiles in smiles_list:
    
    rdMolStandardize.StandardizeSmiles(smiles)
    print(smiles)

    # # Reconstruct the RDKit molecule from the SMILES string.
    mol = Chem.MolFromSmiles(smiles)
    # if mol is None:
    #     print(f"Warning: Invalid SMILES string for {i}: {smiles}")

    chooser = LargestFragmentChooser()
    mol = chooser.choose(mol)

    # Assign the molecule name to the RDKit object.
    # mol.SetProp('_Name', idx)

    # Prepare the ligand with Meeko.
    preparator = MoleculePreparation()

    variants = []

    # Wrap the scrub(mol) in a try-except block

        # Scrubber handles protonation states, 3D coordinates, and tautomers.
    for mol_state in scrub(mol):
        variant_mol_name = f"{i}-{i}"
        
        fragments = Chem.GetMolFrags(mol_state, asMols=True)
        if len(fragments) > 1:
            mol_state = max(fragments, key=lambda m: m.GetNumAtoms())

        print("SMILES: ", smiles, Chem.MolToSmiles(mol_state), "Conformers: %d" % mol_state.GetNumConformers())

        mol_setups = preparator.prepare(mol_state)

        for setup in mol_setups:
            pdbqt_string, is_ok, error_msg = PDBQTWriterLegacy.write_string(setup)
            if is_ok:
                modified_pdbqt = f"REMARK Name = {variant_mol_name.strip()}\n{pdbqt_string}"
                variants.append((variant_mol_name, modified_pdbqt))

    for variant in variants:
        print(variant, sep = "\n")

In [ ]:
rdMolStandardize.StandardizeSmiles("CC(C)NC(=O)C1=CC=CO1")

In [3]:
ligands = [('s_527____153905____156108-0', 'REMARK Name = s_527____153905____156108-0\nREMARK SMILES CC(C)NC(=O)c1ccco1\nREMARK SMILES IDX 4 1 5 2 6 3 2 5 1 6 3 7 7 8 8 9 11 10 9 11 10 12\nREMARK H PARENT 4 4\nROOT\nATOM      1  N   UNL     1      -0.561   0.205  -0.146  1.00  0.00    -0.347 N \nATOM      2  C   UNL     1       0.345  -0.702  -0.658  1.00  0.00     0.287 C \nATOM      3  O   UNL     1       0.002  -1.752  -1.194  1.00  0.00    -0.266 OA\nATOM      4  H   UNL     1      -0.168   0.976   0.384  1.00  0.00     0.164 HD\nENDROOT\nBRANCH   1   5\nATOM      5  C   UNL     1      -1.984  -0.072  -0.063  1.00  0.00     0.076 C \nATOM      6  C   UNL     1      -2.771   1.235  -0.038  1.00  0.00     0.030 C \nATOM      7  C   UNL     1      -2.297  -0.916   1.170  1.00  0.00     0.030 C \nENDBRANCH   1   5\nBRANCH   2   8\nATOM      8  C   UNL     1       1.728  -0.303  -0.493  1.00  0.00     0.191 A \nATOM      9  C   UNL     1       2.904  -0.919  -0.878  1.00  0.00     0.056 A \nATOM     10  O   UNL     1       2.021   0.875   0.149  1.00  0.00    -0.459 OA\nATOM     11  C   UNL     1       3.963  -0.080  -0.455  1.00  0.00     0.043 A \nATOM     12  C   UNL     1       3.374   0.996   0.164  1.00  0.00     0.196 A \nENDBRANCH   2   8\nTORSDOF 2\n'), ('m_22____57676____19882710-0', 'REMARK Name = m_22____57676____19882710-0\nREMARK SMILES COCCNC(=O)C(C)(C)Cl\nREMARK SMILES IDX 4 1 3 2 2 3 1 4 5 5 6 6 7 8 8 9 9 10 10 11 11 12\nREMARK H PARENT 5 7\nROOT\nATOM      1  C   UNL     1       1.386  -0.439  -0.667  1.00  0.00     0.144 C \nENDROOT\nBRANCH   1   2\nATOM      2  C   UNL     1       2.131  -0.325   0.667  1.00  0.00     0.181 C \nBRANCH   2   3\nATOM      3  O   UNL     1       3.484  -0.773   0.553  1.00  0.00    -0.383 OA\nATOM      4  C   UNL     1       4.369   0.239   0.091  1.00  0.00     0.194 C \nENDBRANCH   2   3\nENDBRANCH   1   2\nBRANCH   1   5\nATOM      5  N   UNL     1       0.061   0.152  -0.632  1.00  0.00    -0.352 N \nATOM      6  C   UNL     1      -0.994  -0.468   0.014  1.00  0.00     0.240 C \nATOM      7  H   UNL     1      -0.022   1.149  -0.798  1.00  0.00     0.163 HD\nATOM      8  O   UNL     1      -0.888  -1.590   0.504  1.00  0.00    -0.273 OA\nBRANCH   6   9\nATOM      9  C   UNL     1      -2.317   0.324   0.012  1.00  0.00     0.117 C \nATOM     10  C   UNL     1      -3.346  -0.324   0.937  1.00  0.00     0.039 C \nATOM     11  C   UNL     1      -2.865   0.428  -1.412  1.00  0.00     0.039 C \nATOM     12  Cl  UNL     1      -1.964   1.985   0.622  1.00  0.00    -0.109 Cl\nENDBRANCH   6   9\nENDBRANCH   1   5\nTORSDOF 4\n'), ('m_271302____8904550____25575944-0', 'REMARK Name = m_271302____8904550____25575944-0\nREMARK SMILES COC(CN)CN1CCCC1\nREMARK SMILES IDX 3 1 2 2 1 3 4 4 5 5 6 8 7 9 8 10 11 11 9 12 10 13\nREMARK H PARENT 5 6 5 7\nROOT\nATOM      1  C   UNL     1       1.460  -0.277   0.397  1.00  0.00     0.145 C \nENDROOT\nBRANCH   1   2\nATOM      2  O   UNL     1       2.723   0.425   0.486  1.00  0.00    -0.379 OA\nATOM      3  C   UNL     1       2.764   1.339   1.576  1.00  0.00     0.195 C \nENDBRANCH   1   2\nBRANCH   1   4\nATOM      4  C   UNL     1       1.755  -1.637  -0.276  1.00  0.00     0.111 C \nBRANCH   4   5\nATOM      5  N   UNL     1       2.357  -1.502  -1.610  1.00  0.00    -0.328 NA\nATOM      6  H   UNL     1       3.115  -0.817  -1.527  1.00  0.00     0.118 HD\nATOM      7  H   UNL     1       2.801  -2.382  -1.867  1.00  0.00     0.118 HD\nENDBRANCH   4   5\nENDBRANCH   1   4\nBRANCH   1   8\nATOM      8  C   UNL     1       0.412   0.575  -0.369  1.00  0.00     0.117 C \nBRANCH   8   9\nATOM      9  N   UNL     1      -0.966   0.431   0.135  1.00  0.00    -0.301 NA\nATOM     10  C   UNL     1      -1.869   1.397  -0.499  1.00  0.00     0.084 C \nATOM     11  C   UNL     1      -1.565  -0.874  -0.148  1.00  0.00     0.084 C \nATOM     12  C   UNL     1      -3.243   0.873  -0.119  1.00  0.00     0.017 C \nATOM     13  C   UNL     1      -3.059  -0.633   0.025  1.00  0.00     0.017 C \nENDBRANCH   8   9\nENDBRANCH   1   8\nTORSDOF 5\n'), ('m_271302____8904550____25575944-1', 'REMARK Name = m_271302____8904550____25575944-1\nREMARK SMILES COC(CN)C[NH+]1CCCC1\nREMARK SMILES IDX 3 1 2 2 1 3 4 4 5 5 6 8 7 9 8 10 11 11 9 12 10 13\nREMARK H PARENT 5 6 5 7 7 14\nROOT\nATOM      1  C   UNL     1      -1.153   1.151  -0.741  1.00  0.00     0.186 C \nENDROOT\nBRANCH   1   2\nATOM      2  O   UNL     1      -1.537   1.184   0.644  1.00  0.00    -0.374 OA\nATOM      3  C   UNL     1      -2.815   1.773   0.866  1.00  0.00     0.195 C \nENDBRANCH   1   2\nBRANCH   1   4\nATOM      4  C   UNL     1      -1.863   0.005  -1.503  1.00  0.00     0.115 C \nBRANCH   4   5\nATOM      5  N   UNL     1      -1.560  -1.322  -0.926  1.00  0.00    -0.328 NA\nATOM      6  H   UNL     1      -2.068  -1.380  -0.033  1.00  0.00     0.118 HD\nATOM      7  H   UNL     1      -2.033  -2.027  -1.506  1.00  0.00     0.118 HD\nENDBRANCH   4   5\nENDBRANCH   1   4\nBRANCH   1   8\nATOM      8  C   UNL     1       0.388   1.010  -0.822  1.00  0.00     0.295 C \nBRANCH   8   9\nATOM      9  N   UNL     1       0.875  -0.274  -0.185  1.00  0.00    -0.333 N \nATOM     10  C   UNL     1       2.211  -0.773  -0.673  1.00  0.00     0.262 C \nATOM     11  C   UNL     1       0.903  -0.270   1.327  1.00  0.00     0.262 C \nATOM     12  C   UNL     1       3.044  -0.953   0.584  1.00  0.00     0.067 C \nATOM     13  C   UNL     1       2.018  -1.240   1.660  1.00  0.00     0.067 C \nATOM     14  H   UNL     1       0.152  -0.981  -0.451  1.00  0.00     0.348 HD\nENDBRANCH   8   9\nENDBRANCH   1   8\nTORSDOF 5\n'), ('m_271302____8904550____25575944-2', 'REMARK Name = m_271302____8904550____25575944-2\nREMARK SMILES COC(C[NH3+])CN1CCCC1\nREMARK SMILES IDX 3 1 2 2 1 3 4 4 5 5 6 9 7 10 8 11 11 12 9 13 10 14\nREMARK H PARENT 5 6 5 7 5 8\nROOT\nATOM      1  C   UNL     1       1.327   0.878   0.211  1.00  0.00     0.186 C \nENDROOT\nBRANCH   1   2\nATOM      2  O   UNL     1       2.174   1.935  -0.252  1.00  0.00    -0.374 OA\nATOM      3  C   UNL     1       2.820   2.652   0.795  1.00  0.00     0.195 C \nENDBRANCH   1   2\nBRANCH   1   4\nATOM      4  C   UNL     1       2.153  -0.420   0.332  1.00  0.00     0.292 C \nBRANCH   4   5\nATOM      5  N   UNL     1       1.316  -1.570   0.758  1.00  0.00    -0.355 N \nATOM      6  H   UNL     1       1.061  -1.529   1.750  1.00  0.00     0.344 HD\nATOM      7  H   UNL     1       1.756  -2.476   0.561  1.00  0.00     0.344 HD\nATOM      8  H   UNL     1       0.418  -1.532   0.227  1.00  0.00     0.344 HD\nENDBRANCH   4   5\nENDBRANCH   1   4\nBRANCH   1   9\nATOM      9  C   UNL     1       0.192   0.713  -0.830  1.00  0.00     0.122 C \nBRANCH   9  10\nATOM     10  N   UNL     1      -0.756  -0.367  -0.465  1.00  0.00    -0.301 NA\nATOM     11  C   UNL     1      -1.748   0.100   0.530  1.00  0.00     0.084 C \nATOM     12  C   UNL     1      -1.603  -0.716  -1.628  1.00  0.00     0.084 C \nATOM     13  C   UNL     1      -2.849  -0.945   0.429  1.00  0.00     0.017 C \nATOM     14  C   UNL     1      -2.760  -1.475  -0.996  1.00  0.00     0.017 C \nENDBRANCH   9  10\nENDBRANCH   1   9\nTORSDOF 5\n'), ('m_271302____8904550____25575944-3', 'REMARK Name = m_271302____8904550____25575944-3\nREMARK SMILES COC(C[NH3+])C[NH+]1CCCC1\nREMARK SMILES IDX 3 1 2 2 1 3 4 4 5 5 6 9 7 10 8 11 11 12 9 13 10 14\nREMARK H PARENT 5 6 5 7 5 8 7 15\nROOT\nATOM      1  C   UNL     1      -1.081  -0.979  -0.079  1.00  0.00     0.227 C \nENDROOT\nBRANCH   1   2\nATOM      2  O   UNL     1      -1.520   0.121  -0.891  1.00  0.00    -0.370 OA\nATOM      3  C   UNL     1      -1.272  -0.066  -2.288  1.00  0.00     0.195 C \nENDBRANCH   1   2\nBRANCH   1   4\nATOM      4  C   UNL     1      -2.313  -1.491   0.708  1.00  0.00     0.297 C \nBRANCH   4   5\nATOM      5  N   UNL     1      -3.473  -1.705  -0.206  1.00  0.00    -0.355 N \nATOM      6  H   UNL     1      -3.660  -0.846  -0.749  1.00  0.00     0.344 HD\nATOM      7  H   UNL     1      -3.340  -2.472  -0.879  1.00  0.00     0.344 HD\nATOM      8  H   UNL     1      -4.339  -1.914   0.315  1.00  0.00     0.344 HD\nENDBRANCH   4   5\nENDBRANCH   1   4\nBRANCH   1   9\nATOM      9  C   UNL     1       0.010  -0.519   0.930  1.00  0.00     0.300 C \nBRANCH   9  10\nATOM     10  N   UNL     1       1.212   0.113   0.258  1.00  0.00    -0.333 N \nATOM     11  C   UNL     1       2.467   0.084   1.125  1.00  0.00     0.262 C \nATOM     12  C   UNL     1       1.029   1.573  -0.172  1.00  0.00     0.262 C \nATOM     13  C   UNL     1       3.264   1.270   0.633  1.00  0.00     0.067 C \nATOM     14  C   UNL     1       2.195   2.325   0.447  1.00  0.00     0.067 C \nATOM     15  H   UNL     1       1.455  -0.412  -0.593  1.00  0.00     0.348 HD\nENDBRANCH   9  10\nENDBRANCH   1   9\nTORSDOF 5\n'), ('m_276436____14076548____23408838-0', 'REMARK Name = m_276436____14076548____23408838-0\nREMARK SMILES CC(C)COC(=O)CC(Cl)Cl\nREMARK SMILES IDX 5 1 4 2 2 3 1 4 3 5 6 6 7 7 8 8 9 9 10 10 11 11\nREMARK H PARENT\nROOT\nATOM      1  O   UNL     1       0.408   0.378  -0.974  1.00  0.00    -0.465 OA\nENDROOT\nBRANCH   1   2\nATOM      2  C   UNL     1      -0.989   0.035  -0.984  1.00  0.00     0.235 C \nBRANCH   2   3\nATOM      3  C   UNL     1      -1.551  -0.004   0.443  1.00  0.00     0.021 C \nATOM      4  C   UNL     1      -1.488   1.377   1.099  1.00  0.00     0.011 C \nATOM      5  C   UNL     1      -2.989  -0.519   0.434  1.00  0.00     0.011 C \nENDBRANCH   2   3\nENDBRANCH   1   2\nBRANCH   1   6\nATOM      6  C   UNL     1       1.274  -0.665  -0.849  1.00  0.00     0.308 C \nATOM      7  O   UNL     1       0.953  -1.841  -0.756  1.00  0.00    -0.252 OA\nBRANCH   6   8\nATOM      8  C   UNL     1       2.694  -0.173  -0.867  1.00  0.00     0.158 C \nBRANCH   8   9\nATOM      9  C   UNL     1       3.356  -0.213   0.497  1.00  0.00     0.183 C \nATOM     10  Cl  UNL     1       2.496   0.772   1.705  1.00  0.00    -0.105 Cl\nATOM     11  Cl  UNL     1       3.593  -1.867   1.108  1.00  0.00    -0.105 Cl\nENDBRANCH   8   9\nENDBRANCH   6   8\nENDBRANCH   1   6\nTORSDOF 5\n'), ('s_62____875850____8349548-0', 'REMARK Name = s_62____875850____8349548-0\nREMARK SMILES CSCc1ccc(CF)cc1\nREMARK SMILES IDX 4 1 5 2 6 3 7 4 10 5 11 6 3 7 2 8 1 9 8 10 9 11\nREMARK H PARENT\nROOT\nATOM      1  C   UNL     1       0.383   0.122   0.160  1.00  0.00    -0.038 A \nATOM      2  C   UNL     1      -0.246   1.171  -0.524  1.00  0.00     0.005 A \nATOM      3  C   UNL     1      -1.640   1.251  -0.577  1.00  0.00     0.007 A \nATOM      4  C   UNL     1      -2.432   0.282   0.054  1.00  0.00    -0.017 A \nATOM      5  C   UNL     1      -1.805  -0.773   0.725  1.00  0.00     0.007 A \nATOM      6  C   UNL     1      -0.411  -0.855   0.776  1.00  0.00     0.005 A \nENDROOT\nBRANCH   1   7\nATOM      7  C   UNL     1       1.887   0.030   0.210  1.00  0.00     0.104 C \nBRANCH   7   8\nATOM      8  S   UNL     1       2.553  -0.861  -1.228  1.00  0.00    -0.161 SA\nATOM      9  C   UNL     1       4.308  -0.775  -0.806  1.00  0.00     0.085 C \nENDBRANCH   7   8\nENDBRANCH   1   7\nBRANCH   4  10\nATOM     10  C   UNL     1      -3.931   0.366  -0.024  1.00  0.00     0.250 C \nATOM     11  F   UNL     1      -4.570  -0.534   0.778  1.00  0.00    -0.246 F \nENDBRANCH   4  10\nTORSDOF 3\n')]
ligands2 = [('s_527____153905____156108', 'REMARK Name = s_527____153905____156108\nREMARK SMILES CC(C)NC(=O)c1ccco1\nREMARK SMILES IDX 4 1 5 2 6 3 2 5 1 6 3 7 7 8 8 9 11 10 9 11 10 12\nREMARK H PARENT 4 4\nROOT\nATOM      1  N   UNL     1      -0.529   0.218  -0.294  1.00  0.00    -0.347 N \nATOM      2  C   UNL     1       0.449   0.556   0.629  1.00  0.00     0.287 C \nATOM      3  O   UNL     1       0.161   1.078   1.704  1.00  0.00    -0.266 OA\nATOM      4  H   UNL     1      -0.268  -0.373  -1.069  1.00  0.00     0.164 HD\nENDROOT\nBRANCH   1   5\nATOM      5  C   UNL     1      -1.946   0.354   0.012  1.00  0.00     0.076 C \nATOM      6  C   UNL     1      -2.746   0.494  -1.280  1.00  0.00     0.030 C \nATOM      7  C   UNL     1      -2.438  -0.842   0.825  1.00  0.00     0.030 C \nENDBRANCH   1   5\nBRANCH   2   8\nATOM      8  C   UNL     1       1.808   0.234   0.226  1.00  0.00     0.191 A \nATOM      9  C   UNL     1       2.365  -0.242  -0.950  1.00  0.00     0.056 A \nATOM     10  O   UNL     1       2.810   0.418   1.145  1.00  0.00    -0.459 OA\nATOM     11  C   UNL     1       3.761  -0.356  -0.731  1.00  0.00     0.043 A \nATOM     12  C   UNL     1       3.979   0.058   0.559  1.00  0.00     0.196 A \nENDBRANCH   2   8\nTORSDOF 2\n'), ('m_22____57676____19882710', 'REMARK Name = m_22____57676____19882710\nREMARK SMILES COCCNC(=O)C(C)(C)Cl\nREMARK SMILES IDX 4 1 3 2 2 3 1 4 5 5 6 6 7 8 8 9 9 10 10 11 11 12\nREMARK H PARENT 5 7\nROOT\nATOM      1  C   UNL     1       1.052   1.195  -0.721  1.00  0.00     0.144 C \nENDROOT\nBRANCH   1   2\nATOM      2  C   UNL     1       2.222   0.267  -1.038  1.00  0.00     0.181 C \nBRANCH   2   3\nATOM      3  O   UNL     1       2.522  -0.512   0.128  1.00  0.00    -0.383 OA\nATOM      4  C   UNL     1       3.631  -1.374  -0.096  1.00  0.00     0.194 C \nENDBRANCH   2   3\nENDBRANCH   1   2\nBRANCH   1   5\nATOM      5  N   UNL     1      -0.110   0.439  -0.284  1.00  0.00    -0.352 N \nATOM      6  C   UNL     1      -1.368   1.017  -0.254  1.00  0.00     0.240 C \nATOM      7  H   UNL     1       0.098  -0.333   0.339  1.00  0.00     0.163 HD\nATOM      8  O   UNL     1      -1.562   2.137  -0.724  1.00  0.00    -0.273 OA\nBRANCH   6   9\nATOM      9  C   UNL     1      -2.466   0.152   0.409  1.00  0.00     0.117 C \nATOM     10  C   UNL     1      -2.578  -1.208  -0.284  1.00  0.00     0.039 C \nATOM     11  C   UNL     1      -2.180  -0.018   1.903  1.00  0.00     0.039 C \nATOM     12  Cl  UNL     1      -4.055   0.987   0.240  1.00  0.00    -0.109 Cl\nENDBRANCH   6   9\nENDBRANCH   1   5\nTORSDOF 4\n'), ('m_271302____8904550____25575944', 'REMARK Name = m_271302____8904550____25575944\nREMARK SMILES COC(CN)CN1CCCC1\nREMARK SMILES IDX 3 1 2 2 1 3 4 4 5 5 6 8 7 9 8 10 11 11 9 12 10 13\nREMARK H PARENT 5 6 5 7\nROOT\nATOM      1  C   UNL     1      -1.453  -0.167   0.498  1.00  0.00     0.145 C \nENDROOT\nBRANCH   1   2\nATOM      2  O   UNL     1      -2.712   0.535   0.379  1.00  0.00    -0.379 OA\nATOM      3  C   UNL     1      -2.728   1.777   1.070  1.00  0.00     0.195 C \nENDBRANCH   1   2\nBRANCH   1   4\nATOM      4  C   UNL     1      -1.748  -1.672   0.292  1.00  0.00     0.111 C \nBRANCH   4   5\nATOM      5  N   UNL     1      -2.456  -1.956  -0.966  1.00  0.00    -0.328 NA\nATOM      6  H   UNL     1      -2.876  -2.881  -0.911  1.00  0.00     0.118 HD\nATOM      7  H   UNL     1      -3.228  -1.288  -1.032  1.00  0.00     0.118 HD\nENDBRANCH   4   5\nENDBRANCH   1   4\nBRANCH   1   8\nATOM      8  C   UNL     1      -0.414   0.375  -0.515  1.00  0.00     0.117 C \nBRANCH   8   9\nATOM      9  N   UNL     1       0.942  -0.164  -0.328  1.00  0.00    -0.301 NA\nATOM     10  C   UNL     1       1.827   0.198  -1.437  1.00  0.00     0.084 C \nATOM     11  C   UNL     1       1.614   0.342   0.870  1.00  0.00     0.084 C \nATOM     12  C   UNL     1       3.206  -0.136  -0.892  1.00  0.00     0.017 C \nATOM     13  C   UNL     1       3.079   0.012   0.620  1.00  0.00     0.017 C \nENDBRANCH   8   9\nENDBRANCH   1   8\nTORSDOF 5\n'), ('m_271302____8904550____25575944', 'REMARK Name = m_271302____8904550____25575944\nREMARK SMILES COC(CN)C[NH+]1CCCC1\nREMARK SMILES IDX 3 1 2 2 1 3 4 4 5 5 6 8 7 9 8 10 11 11 9 12 10 13\nREMARK H PARENT 5 6 5 7 7 14\nROOT\nATOM      1  C   UNL     1      -1.403   0.597  -0.490  1.00  0.00     0.186 C \nENDROOT\nBRANCH   1   2\nATOM      2  O   UNL     1      -1.088   1.301   0.734  1.00  0.00    -0.374 OA\nATOM      3  C   UNL     1      -2.235   1.805   1.417  1.00  0.00     0.195 C \nENDBRANCH   1   2\nBRANCH   1   4\nATOM      4  C   UNL     1      -2.126  -0.752  -0.263  1.00  0.00     0.115 C \nBRANCH   4   5\nATOM      5  N   UNL     1      -3.551  -0.591   0.020  1.00  0.00    -0.328 NA\nATOM      6  H   UNL     1      -3.975  -1.497   0.230  1.00  0.00     0.118 HD\nATOM      7  H   UNL     1      -4.053  -0.237  -0.795  1.00  0.00     0.118 HD\nENDBRANCH   4   5\nENDBRANCH   1   4\nBRANCH   1   8\nATOM      8  C   UNL     1      -0.041   0.393  -1.195  1.00  0.00     0.295 C \nBRANCH   8   9\nATOM      9  N   UNL     1       1.062   0.212  -0.172  1.00  0.00    -0.333 N \nATOM     10  C   UNL     1       1.185  -1.154   0.452  1.00  0.00     0.262 C \nATOM     11  C   UNL     1       2.439   0.641  -0.624  1.00  0.00     0.262 C \nATOM     12  C   UNL     1       2.647  -1.234   0.841  1.00  0.00     0.067 C \nATOM     13  C   UNL     1       3.338  -0.542  -0.315  1.00  0.00     0.067 C \nATOM     14  H   UNL     1       0.768   0.853   0.591  1.00  0.00     0.348 HD\nENDBRANCH   8   9\nENDBRANCH   1   8\nTORSDOF 5\n'), ('m_271302____8904550____25575944', 'REMARK Name = m_271302____8904550____25575944\nREMARK SMILES COC(C[NH3+])CN1CCCC1\nREMARK SMILES IDX 3 1 2 2 1 3 4 4 5 5 6 9 7 10 8 11 11 12 9 13 10 14\nREMARK H PARENT 5 6 5 7 5 8\nROOT\nATOM      1  C   UNL     1      -1.346   0.258  -0.142  1.00  0.00     0.186 C \nENDROOT\nBRANCH   1   2\nATOM      2  O   UNL     1      -2.554   0.857   0.379  1.00  0.00    -0.374 OA\nATOM      3  C   UNL     1      -2.515   2.284   0.403  1.00  0.00     0.195 C \nENDBRANCH   1   2\nBRANCH   1   4\nATOM      4  C   UNL     1      -1.643  -1.250  -0.230  1.00  0.00     0.292 C \nBRANCH   4   5\nATOM      5  N   UNL     1      -3.014  -1.425  -0.798  1.00  0.00    -0.355 N \nATOM      6  H   UNL     1      -3.380  -2.372  -0.655  1.00  0.00     0.344 HD\nATOM      7  H   UNL     1      -3.053  -1.190  -1.795  1.00  0.00     0.344 HD\nATOM      8  H   UNL     1      -3.627  -0.754  -0.303  1.00  0.00     0.344 HD\nENDBRANCH   4   5\nENDBRANCH   1   4\nBRANCH   1   9\nATOM      9  C   UNL     1      -0.112   0.502   0.752  1.00  0.00     0.122 C \nBRANCH   9  10\nATOM     10  N   UNL     1       1.046  -0.285   0.320  1.00  0.00    -0.301 NA\nATOM     11  C   UNL     1       2.113  -0.279   1.333  1.00  0.00     0.084 C \nATOM     12  C   UNL     1       1.699   0.251  -0.883  1.00  0.00     0.084 C \nATOM     13  C   UNL     1       3.319  -0.804   0.570  1.00  0.00     0.017 C \nATOM     14  C   UNL     1       3.048  -0.452  -0.888  1.00  0.00     0.017 C \nENDBRANCH   9  10\nENDBRANCH   1   9\nTORSDOF 5\n'), ('m_271302____8904550____25575944', 'REMARK Name = m_271302____8904550____25575944\nREMARK SMILES COC(C[NH3+])C[NH+]1CCCC1\nREMARK SMILES IDX 3 1 2 2 1 3 4 4 5 5 6 9 7 10 8 11 11 12 9 13 10 14\nREMARK H PARENT 5 6 5 7 5 8 7 15\nROOT\nATOM      1  C   UNL     1       1.283  -0.624  -0.356  1.00  0.00     0.227 C \nENDROOT\nBRANCH   1   2\nATOM      2  O   UNL     1       1.539   0.498   0.501  1.00  0.00    -0.370 OA\nATOM      3  C   UNL     1       2.136   1.609  -0.177  1.00  0.00     0.195 C \nENDBRANCH   1   2\nBRANCH   1   4\nATOM      4  C   UNL     1       2.114  -1.819   0.176  1.00  0.00     0.297 C \nBRANCH   4   5\nATOM      5  N   UNL     1       3.534  -1.426   0.404  1.00  0.00    -0.355 N \nATOM      6  H   UNL     1       3.581  -0.602   1.026  1.00  0.00     0.344 HD\nATOM      7  H   UNL     1       4.068  -2.174   0.874  1.00  0.00     0.344 HD\nATOM      8  H   UNL     1       4.048  -1.193  -0.457  1.00  0.00     0.344 HD\nENDBRANCH   4   5\nENDBRANCH   1   4\nBRANCH   1   9\nATOM      9  C   UNL     1      -0.224  -1.008  -0.325  1.00  0.00     0.300 C \nBRANCH   9  10\nATOM     10  N   UNL     1      -1.148   0.125  -0.735  1.00  0.00    -0.333 N \nATOM     11  C   UNL     1      -1.357   1.200   0.333  1.00  0.00     0.262 C \nATOM     12  C   UNL     1      -2.550  -0.353  -1.136  1.00  0.00     0.262 C \nATOM     13  C   UNL     1      -2.802   1.619   0.165  1.00  0.00     0.067 C \nATOM     14  C   UNL     1      -3.493   0.314  -0.155  1.00  0.00     0.067 C \nATOM     15  H   UNL     1      -0.770   0.602  -1.565  1.00  0.00     0.348 HD\nENDBRANCH   9  10\nENDBRANCH   1   9\nTORSDOF 5\n'), ('m_276436____14076548____23408838', 'REMARK Name = m_276436____14076548____23408838\nREMARK SMILES CC(C)COC(=O)CC(Cl)Cl\nREMARK SMILES IDX 5 1 4 2 2 3 1 4 3 5 6 6 7 7 8 8 9 9 10 10 11 11\nREMARK H PARENT\nROOT\nATOM      1  O   UNL     1       0.408  -1.058   0.233  1.00  0.00    -0.465 OA\nENDROOT\nBRANCH   1   2\nATOM      2  C   UNL     1      -0.917  -1.110  -0.323  1.00  0.00     0.235 C \nBRANCH   2   3\nATOM      3  C   UNL     1      -1.856  -0.195   0.473  1.00  0.00     0.021 C \nATOM      4  C   UNL     1      -3.298  -0.375   0.000  1.00  0.00     0.011 C \nATOM      5  C   UNL     1      -1.444   1.274   0.362  1.00  0.00     0.011 C \nENDBRANCH   2   3\nENDBRANCH   1   2\nBRANCH   1   6\nATOM      6  C   UNL     1       1.372  -0.465  -0.522  1.00  0.00     0.308 C \nATOM      7  O   UNL     1       1.214   0.006  -1.639  1.00  0.00    -0.252 OA\nBRANCH   6   8\nATOM      8  C   UNL     1       2.689  -0.516   0.196  1.00  0.00     0.158 C \nBRANCH   8   9\nATOM      9  C   UNL     1       3.517   0.704  -0.163  1.00  0.00     0.183 C \nATOM     10  Cl  UNL     1       2.846   2.182   0.585  1.00  0.00    -0.105 Cl\nATOM     11  Cl  UNL     1       5.192   0.484   0.426  1.00  0.00    -0.105 Cl\nENDBRANCH   8   9\nENDBRANCH   6   8\nENDBRANCH   1   6\nTORSDOF 5\n'), ('s_62____875850____8349548', 'REMARK Name = s_62____875850____8349548\nREMARK SMILES CSCc1ccc(CF)cc1\nREMARK SMILES IDX 4 1 5 2 6 3 7 4 10 5 11 6 3 7 2 8 1 9 8 10 9 11\nREMARK H PARENT\nROOT\nATOM      1  C   UNL     1      -0.364   0.164   0.161  1.00  0.00    -0.038 A \nATOM      2  C   UNL     1       0.437   1.145  -0.438  1.00  0.00     0.005 A \nATOM      3  C   UNL     1       1.827   1.006  -0.470  1.00  0.00     0.007 A \nATOM      4  C   UNL     1       2.443  -0.118   0.096  1.00  0.00    -0.017 A \nATOM      5  C   UNL     1       1.642  -1.105   0.682  1.00  0.00     0.007 A \nATOM      6  C   UNL     1       0.252  -0.967   0.713  1.00  0.00     0.005 A \nENDROOT\nBRANCH   1   7\nATOM      7  C   UNL     1      -1.864   0.310   0.189  1.00  0.00     0.104 C \nBRANCH   7   8\nATOM      8  S   UNL     1      -2.632  -0.360  -1.317  1.00  0.00    -0.161 SA\nATOM      9  C   UNL     1      -4.360  -0.026  -0.907  1.00  0.00     0.085 C \nENDBRANCH   7   8\nENDBRANCH   1   7\nBRANCH   4  10\nATOM     10  C   UNL     1       3.938  -0.270   0.040  1.00  0.00     0.250 C \nATOM     11  F   UNL     1       4.408  -1.314   0.781  1.00  0.00    -0.246 F \nENDBRANCH   4  10\nTORSDOF 3\n')]

for ligand in ligands2:
    print(ligand)

('s_527____153905____156108', 'REMARK Name = s_527____153905____156108\nREMARK SMILES CC(C)NC(=O)c1ccco1\nREMARK SMILES IDX 4 1 5 2 6 3 2 5 1 6 3 7 7 8 8 9 11 10 9 11 10 12\nREMARK H PARENT 4 4\nROOT\nATOM      1  N   UNL     1      -0.529   0.218  -0.294  1.00  0.00    -0.347 N \nATOM      2  C   UNL     1       0.449   0.556   0.629  1.00  0.00     0.287 C \nATOM      3  O   UNL     1       0.161   1.078   1.704  1.00  0.00    -0.266 OA\nATOM      4  H   UNL     1      -0.268  -0.373  -1.069  1.00  0.00     0.164 HD\nENDROOT\nBRANCH   1   5\nATOM      5  C   UNL     1      -1.946   0.354   0.012  1.00  0.00     0.076 C \nATOM      6  C   UNL     1      -2.746   0.494  -1.280  1.00  0.00     0.030 C \nATOM      7  C   UNL     1      -2.438  -0.842   0.825  1.00  0.00     0.030 C \nENDBRANCH   1   5\nBRANCH   2   8\nATOM      8  C   UNL     1       1.808   0.234   0.226  1.00  0.00     0.191 A \nATOM      9  C   UNL     1       2.365  -0.242  -0.950  1.00  0.00     0.056 A \nATOM     1